In [1]:
import os
current_dir = os.getcwd()

import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType

In [2]:
spark = SparkSession \
    .builder \
    .appName("structured") \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble
23/12/23 13:38:33 WARN Utils: Your hostname, dam resolves to a loopback address: 127.0.1.1; using 172.29.238.84 instead (on interface eth0)
23/12/23 13:38:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/23 13:38:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data_path = f"{current_dir}/data/lematized"

In [4]:
user_schema = StructType([
    StructField("date", TimestampType()),
    StructField("channel_id", IntegerType()),
    StructField("text", StringType()),
    StructField("media", StringType()),
])

In [5]:
streaming_df = spark \
    .readStream \
    .schema(user_schema) \
    .parquet(data_path) \
    .withWatermark("date", "10 minutes")

In [6]:
words = (streaming_df \
    .select(streaming_df.date, F.explode(F.split(streaming_df.text, " ")) \
    .alias("word"))) \
    .filter(F.length("word") > 3)

In [7]:
word_counts = words \
    .groupBy(F.window(words.date, "10 minutes"), "word") \
    .count() \
    .orderBy(F.desc("window"), F.desc("count"))

In [8]:
query = word_counts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+-----------------+-----+
|              window|             word|count|
+--------------------+-----------------+-----+
|{2023-12-17 23:30...|         временно|    1|
|{2023-12-17 23:30...|        коррупция|    1|
|{2023-12-17 23:30...|           запись|    1|
|{2023-12-17 23:30...|          спонсор|    1|
|{2023-12-17 23:30...|        исключить|    1|
|{2023-12-17 23:30...|        агентство|    1|
|{2023-12-17 23:30...|    международный|    1|
|{2023-12-17 23:30...| попредотвращение|    1|
|{2023-12-17 23:30...|          украина|    1|
|{2023-12-17 23:30...|     национальный|    1|
|{2023-12-17 23:30...|           насайт|    1|
|{2023-12-17 23:30...|           группа|    1|
|{2023-12-17 23:30...|         изсписок|    1|
|{2023-12-17 23:30...|            война|    1|
|{2023-12-17 23:30...|свидетельствовать|    1|
|{2023-12-17 23:30...|       банковский|    1|
|{2023-12-

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----------------+-----+
|              window|             word|count|
+--------------------+-----------------+-----+
|{2023-12-23 13:30...|             тест|    2|
|{2023-12-23 13:30...|           привет|    1|
|{2023-12-23 13:30...|         проверка|    1|
|{2023-12-17 23:30...|         временно|    1|
|{2023-12-17 23:30...|        коррупция|    1|
|{2023-12-17 23:30...|           запись|    1|
|{2023-12-17 23:30...|          спонсор|    1|
|{2023-12-17 23:30...|        исключить|    1|
|{2023-12-17 23:30...|        агентство|    1|
|{2023-12-17 23:30...|    международный|    1|
|{2023-12-17 23:30...| попредотвращение|    1|
|{2023-12-17 23:30...|          украина|    1|
|{2023-12-17 23:30...|     национальный|    1|
|{2023-12-17 23:30...|           насайт|    1|
|{2023-12-17 23:30...|           группа|    1|
|{2023-12-17 23:30...|         изсписок|    1|
|{2023-12-

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----------------+-----+
|              window|             word|count|
+--------------------+-----------------+-----+
|{2023-12-23 13:30...|             тест|    6|
|{2023-12-23 13:30...|           привет|    1|
|{2023-12-23 13:30...|         проверка|    1|
|{2023-12-17 23:30...|         временно|    1|
|{2023-12-17 23:30...|        коррупция|    1|
|{2023-12-17 23:30...|           запись|    1|
|{2023-12-17 23:30...|          спонсор|    1|
|{2023-12-17 23:30...|        исключить|    1|
|{2023-12-17 23:30...|        агентство|    1|
|{2023-12-17 23:30...|    международный|    1|
|{2023-12-17 23:30...| попредотвращение|    1|
|{2023-12-17 23:30...|          украина|    1|
|{2023-12-17 23:30...|     национальный|    1|
|{2023-12-17 23:30...|           насайт|    1|
|{2023-12-17 23:30...|           группа|    1|
|{2023-12-17 23:30...|         изсписок|    1|
|{2023-12-

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+----------------+-----+
|              window|            word|count|
+--------------------+----------------+-----+
|{2023-12-23 13:40...|          привет|   30|
|{2023-12-23 13:40...|         зюганов|    1|
|{2023-12-23 13:40...|           съезд|    1|
|{2023-12-23 13:40...|      поддержать|    1|
|{2023-12-23 13:40...|       харитонов|    1|
|{2023-12-23 13:40...|        кандидат|    1|
|{2023-12-23 13:40...|       президент|    1|
|{2023-12-23 13:40...|      выдвижение|    1|
|{2023-12-23 13:40...|            кпрф|    1|
|{2023-12-23 13:30...|            тест|    6|
|{2023-12-23 13:30...|          привет|    1|
|{2023-12-23 13:30...|        проверка|    1|
|{2023-12-17 23:30...|        временно|    1|
|{2023-12-17 23:30...|       коррупция|    1|
|{2023-12-17 23:30...|          запись|    1|
|{2023-12-17 23:30...|         спонсор|    1|
|{2023-12-17 23:30...|       

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+----------+-----+
|              window|      word|count|
+--------------------+----------+-----+
|{2023-12-23 13:40...|    привет|   30|
|{2023-12-23 13:40...|    яблоко|    2|
|{2023-12-23 13:40...| президент|    2|
|{2023-12-23 13:40...|    партия|    2|
|{2023-12-23 13:40...|    момент|    1|
|{2023-12-23 13:40...|   зюганов|    1|
|{2023-12-23 13:40...|     съезд|    1|
|{2023-12-23 13:40...|    выборы|    1|
|{2023-12-23 13:40...| екатерина|    1|
|{2023-12-23 13:40...|поддержать|    1|
|{2023-12-23 13:40...|    россия|    1|
|{2023-12-23 13:40...|      быть|    1|
|{2023-12-23 13:40...|обратиться|    1|
|{2023-12-23 13:40...|     никто|    1|
|{2023-12-23 13:40...|    данный|    1|
|{2023-12-23 13:40...|   просьба|    1|
|{2023-12-23 13:40...| харитонов|    1|
|{2023-12-23 13:40...|  кандидат|    1|
|{2023-12-23 13:40...| выдвигать|    1|
|{2023-12-23 13:40...|в